In [138]:

from typing import Type, TypedDict, Union, Optional
from enum import Enum
from datetime import datetime
from pydantic import BaseModel, validator, root_validator

class BatteryManufacturer(str, Enum):  # we may want to allow custom manufacturers
    bigmap = 'BIG-MAP'
    conrad = 'Conrad energy'
    maxell = 'Maxell'

class BatteryComposition(BaseModel):
    description: str = None
    cathode: str = None
    anode: str = None
    electrolyte: str = None
    
    # check that components are not specified if string description was specified
    # build components from string description, or vice versa
    @root_validator
    def validate_composition(cls, values):
        if values['description']:
            if any([values[key] for key in ('cathode', 'anode', 'electrolyte')]):
                raise ValueError("You cannot specify a 'description' and any component at the same time.")
            components = values['description'].split('|')
            if len(components) == 3:
                values['cathode'], values['electrolyte'], values['anode'] = components
            else:
                raise ValueError("Composition 'description' does not have 3 components (i.e. {cathode}|{electrolyte}|{anode}).")
        elif any([values[key] for key in ('cathode', 'anode', 'electrolyte')]):
            values['description'] = f"{values['cathode']}|{values['electrolyte']}|{values['anode']}"
        else:
            raise ValueError("You must specify either a string 'description' or the components.")
        return values
    
class BatteryCapacity(TypedDict): # TypedDict?
    nominal: float
    actual: float

class BatteryMetadata(BaseModel):
    creation_datetime = datetime
    creation_process = str
    
class BatterySpecs(BaseModel):
    """
    Battery specification schema.
    """
    manufacturer: BatteryManufacturer
    composition: Type[BatteryComposition]
    form_factor: Union[int, str]
    capacity: Type[BatteryCapacity]
    metadata: Type[BatteryMetadata]

In [13]:
m = BatteryManufacturer.bigmap

In [56]:
BatteryComposition.schema()

{'title': 'BatteryComposition',
 'type': 'object',
 'properties': {'description': {'title': 'Description', 'type': 'string'},
  'cathode': {'title': 'Cathode', 'type': 'string'},
  'anode': {'title': 'Anode', 'type': 'string'},
  'electrolyte': {'title': 'Electrolyte', 'type': 'string'}},
 'required': ['description']}

In [75]:
if any([1, None]):
    print('yes')

yes


In [135]:
comp = BatteryComposition(anode='A')

In [134]:
comp

{'anode': 'A'}

['Lio', 'Ciao', 'oxi']

In [4]:
BatteryManufacturer

[<enum 'BatteryManufacturer'>, str, <enum 'Enum'>, object]